In [1]:
#!pip install mpire
from function import multiprocess_func, multiprocess_model, slow_function, DummyModel
import pandas as pd

In [9]:
# Reload (if updates occur in function.py module)
import function 
from importlib import reload
reload(function)

from function import multiprocess_func, multiprocess_model, slow_function, DummyModel

In [2]:
# Example use multiprocessing a slow function
sweep = multiprocess_func(slow_function)

sweep(range(50))

[0,
 2,
 4,
 6,
 8,
 10,
 12,
 14,
 16,
 18,
 20,
 22,
 24,
 26,
 28,
 30,
 32,
 34,
 36,
 38,
 40,
 42,
 44,
 46,
 48,
 50,
 52,
 54,
 56,
 58,
 60,
 62,
 64,
 66,
 68,
 70,
 72,
 74,
 76,
 78,
 80,
 82,
 84,
 86,
 88,
 90,
 92,
 94,
 96,
 98]

In [7]:
# Example use mutliprocessing model parameter sweep

# Creating runs for parameter sweep
data = {
    'run_number': range(50),
    'x': range(0, 100, 2),
    'y': range(1, 101, 2)
}
runs_df = pd.DataFrame(data)

# Creating sample data to compare runs to
data = {
    'x': range(0, 400, 8),
    'y': range(200, 400, 4)
}
reference_df = pd.DataFrame(data)
# Compute comparison stats once for all sweeps
# Note: reference_data could include more than one statistic
reference_data = reference_df.mean().to_dict()


# Create an ititalized model for each worker to load
model = DummyModel(reference_data)

sweep = multiprocess_model(model)
output = sweep(runs_df)

pd.DataFrame(output, columns=['x', 'y'])

,x,y
0,-196.0,-296.0
1,-192.0,-292.0
2,-188.0,-288.0
3,-184.0,-284.0
4,-180.0,-280.0
5,-176.0,-276.0
6,-172.0,-272.0
7,-168.0,-268.0
8,-164.0,-264.0
9,-160.0,-260.0


In [10]:
import time
# Each model ~2s and running 50 models so single CPU ~100s
start = time.time()
sweep(runs_df)
end = time.time()
print(end - start)

62.600199460983276


In [ ]:
def yearly_change(df, func='mean', bins=np.arange(2.5, 9)):
    '''
    Returns the rate change (or slopes) of a data that are binned
    where the first value is just the intercept.
    '''
    df = df.stack().copy()
    # Age groups each row should be binned to
    age_groups = pd.cut(df['age_days']/365, bins, labels=False)
    # Bin data, agg with function specified, and get row to row difference
    grouped = df.drop('age_days', axis=1).groupby(age_groups).agg(func)
    # Update index
    grouped.index = (bins[:-1] + 0.5).astype(str)
    slopes = grouped.diff()
    # Add intercept to first value
    slopes.iloc[0] = grouped.iloc[0]

    return slopes.unstack()


grow_df = 